In [1]:
import os

In [3]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

### Clone Github repositories

In [4]:
%pwd

'c:\\learning\\gen_ai\\code-analysis\\GenAI_Source_code_analysis\\research'

In [5]:
!mkdir test_repo

In [ ]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/entbappy/End-to-end-ML-Project-Implementation", to_path=repo_path)

In [ ]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject',
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [ ]:
documents = loader.load()

In [ ]:
documents

### Chunkings

In [ ]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [ ]:
texts = documents_splitter.split_documents(documents)

In [ ]:
len(texts)

### Embedding model

In [ ]:
os.environ["OPENAI_API_KEY"] = "***************************"

In [ ]:
embeddings=OpenAIEmbeddings(disallowed_special=())

### Knowledge base (vector DB)

In [ ]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

### LLM Wrapper

In [ ]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

In [ ]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [ ]:
a = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

### Q&A

In [ ]:
question = "what is DataIngestion class?"

In [ ]:
result = qa(question)
print(result['answer'])